### Imports

In [1]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.inception import model_urls
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# import sparseconvnet.legacy as scn

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"

env: TMP="/tmp"
env: JOBLIB_TEMP_FOLDER="/tmp"


In [2]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [3]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
img_scale = 840
img_size = 640
n_channels = 1

use_gpu = torch.cuda.is_available()

### Dataset class

In [4]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, img_size, transform=None, mode='train'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype
        self.img_size = img_size
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        self.training = True if mode == 'train' else False

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
        
        # randomize thresholding
        if self.training:
            img = img.point(lambda x: x if x < np.random.randint(180,220) else 0)
        else:
            img = img.point(lambda x: x if x < 200 else 0)
            
        # resizing
#         print('PIL size:', img.size)
        img = np.asarray(img)
#         print('Np size: ',img.shape)
        if img.shape[1] > img.shape[0]:
#             print(img.shape)
            img = np.swapaxes(img,0,1)
        img = cv2.resize(img, (img_scale,img_scale+210))
        img = img[210:,:]
#         print(img.shape)

        # rotation
        if self.training:
            rot_ang = range(0,360,30)
            rot_ang = rot_ang[np.random.randint(0,len(rot_ang))]
    #         print(rot_ang)
            scale = np.random.uniform(0.8,1.2)
    #         print(scale)
            rot_mat = cv2.getRotationMatrix2D((img.shape[0]/2,img.shape[1]/2), rot_ang, scale);
            img = cv2.warpAffine(img, rot_mat, (img.shape[0],img.shape[1]))
        
        # randomize thresholding
        if self.training:
            thresh_block_size = np.random.randint(5,9) // 2 * 2 + 1
            thresh_param = np.random.randint(5,9)
        else:
            thresh_block_size = 7
            thresh_param = 7
        
        img_he = self.clahe.apply(cv2.convertScaleAbs(img[:,:,0]))
        img_mask = cv2.adaptiveThreshold(img_he, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,
                                         thresh_block_size, thresh_param)
        img_ch = img_he * img_mask
        
        if n_channels == 1:
            img = img_ch
        elif n_channels == 3:
            for i in xrange(img.shape[2]):
                img[:,:,i] = img_ch
        else:
            print('\n Wrong number of channels \n')

        img = Image.fromarray(np.uint8(img))
        
#         image = cv2.imread(img_name)[:,:,::-1]
        boneage = self.labels.iloc[idx]['boneage'].astype(np.float32)
        gender = self.labels.iloc[idx]['male'].astype(np.uint16)
        id = self.labels.iloc[idx]['id']
        
        label = np.hstack((boneage, gender, id))
#         print(label)

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [5]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.Scale(300),
#         transforms.CenterCrop(384),
        transforms.RandomSizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
#         transforms.Scale(300),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [6]:
labels = pd.read_csv(TRAIN_LABEL, dtype=np.uint16)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [7]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage', 'male')],
        'val': labels.loc[test, ('id', 'boneage', 'male')]
    }
    splits.append(fold_label)

/home/user/software/miniconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


In [8]:
fold_label = splits[0]

In [9]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, img_size, data_transforms[x], x)
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=24, shuffle=True, num_workers=24, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [10]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
#     for i in xrange(sample['image'].size()[0]):
#         img = sample['image'][i,0,:,:].numpy()
#         plt.figure()
#         plt.imshow(img, cmap=plt.cm.gray)
    if i_batch == 0:
        break

0 torch.Size([24, 1, 640, 640])
0 0.257121661431


### Training

In [11]:
model_urls['inception_v3_google'] = model_urls['inception_v3_google'].replace('https://', 'http://')

class AvgPool(nn.Module):
    def forward(self, x):
        x = x.view
        return F.avg_pool2d(x, (x.size(2), x.size(3)))

class Net(nn.Module):
    def __init__(self, input_shape=(n_channels, img_size, img_size)):
        super(Net, self).__init__()
        
#         # inception_v3
#         self.bn = nn.BatchNorm2d(input_shape[0])
#         resnet = models.inception_v3(pretrained=True)
        
#         num_ftrs = resnet.fc.in_features
#         resnet.fc = nn.Linear(num_ftrs, 1)
        
#         num_ftrs_aux = resnet.AuxLogits.fc.in_features
#         resnet.AuxLogits.fc = nn.Linear(num_ftrs_aux, 1)
        
#         self.resnet = resnet
        
#         # resnet
#         self.bn = nn.BatchNorm2d(input_shape[0])
#         resnet = models.resnet50(pretrained=True)
#         num_ftrs = resnet.fc.in_features
#         resnet.fc = nn.Linear(num_ftrs, 1)
#         self.resnet = resnet

        self.net = nn.Sequential(
            nn.Conv2d(input_shape[0], 192, 5, stride=3),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 160, 1, stride=1),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True),
            nn.Conv2d(160, 96, 1, stride=1),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(3, stride=2, ceil_mode=True),
            nn.Dropout(),
            
            nn.Conv2d(96, 192, 5, stride=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            
            nn.AvgPool2d(3, stride=2, ceil_mode=True),
            nn.Dropout(),
        
            nn.Conv2d(192, 192, 3, stride=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, 1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 1, 1, stride=1),
            nn.BatchNorm2d(1),
            nn.ReLU(inplace=True)
                                 
#             nn.AvgPool2d(12, stride=12, ceil_mode=True)
        )
        self.boneage_clf = nn.Sequential(
            nn.Dropout(),
            nn.Linear(12*12, 96),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(96, 48),
            nn.ReLU(inplace=True),
            nn.Linear(48, 1)
        )
        self.gender_clf = nn.Sequential(
            nn.Dropout(),
            nn.Linear(12*12, 96),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(96, 48),
            nn.ReLU(inplace=True),
            nn.Linear(48, 1)
        )
        
#         resnet.avgpool = nn.AdaptiveAvgPool2d(2)
#         num_ftrs = net.fc.in_features
#         net.fc = nn.Linear(num_ftrs, 1)
#         self.net = net

    def forward(self, x):
#         x = self.net(self.bn(x))
        x = self.net(x)
#         print(x.size())
        x = x.view(-1, 12*12)
        g_x = self.gender_clf(x)
        ba_x = self.boneage_clf(x)
#         x = F.relu(self.lin128(F.dropout(x)))
#         x = self.lin1(F.dropout(x))
#         x = self.resnet(self.bn(x))
#         return ba_x
        return ba_x, g_x
    
model = Net([n_channels, img_size, img_size])
    
# print(summary([3, 640, 640], model))
print(model)

if use_gpu:
    model.cuda()

Net (
  (net): Sequential (
    (0): Conv2d(1, 192, kernel_size=(5, 5), stride=(3, 3))
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
    (4): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
    (7): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (10): Dropout (p = 0.5)
    (11): Conv2d(96, 192, kernel_size=(5, 5), stride=(2, 2))
    (12): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (13): ReLU (inplace)
    (14): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (15): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (16): ReLU (inplace)
    (17): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (18): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True)
    (1

In [12]:
lr = 0.01
# momentum = 0.9
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [13]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    exp_lr_scheduler.step()
    model.train()
    train_boneage_loss = 0
    train_gender_loss = 0
    train_loss = 0
    train_ccc = 0
    num_samp = len(train_loader.dataset)
    
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        
#         ba_output = model(data)
        
        # 2 heads MTL output
        ba_output, g_output = model(data)
#         print(output1)
#         print(output2)
        
#         for i in xrange(output.size[0]):
#             if output[i,1] > 0.5:
#                 output[i,0] = (output[i,0] // 6) * 6
#             else:
#                 output[i,0] = (output[i,0] // 2) * 2

#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]

#         boneage_loss = F.l1_loss(output.view(-1), target[:,0])
#         train_boneage_loss += boneage_loss.data[0]
        boneage_loss = F.smooth_l1_loss(ba_output.view(-1), target[:,0])
        train_boneage_loss += boneage_loss.data[0]
        
        gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,1])
        train_gender_loss += gender_loss.data[0]
        
        loss = 0.75 * boneage_loss + 0.25 * gender_loss
#         loss = boneage_loss
#         loss = gender_loss
#         loss = F.l1_loss(output1.view(-1), target[:,0]) + F.l1_loss(output2.view(-1), target[:,0])
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
#         loss = F.l1_loss(output.view(-1), target[:,0])
        
        train_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(ba_output, target[:,0])
        train_ccc += c.data.cpu().numpy()[0]
        
        loss.backward()
        optimizer.step()
#         if batch_idx % (1000 // len(target)) == 0:
    print('Train ep: {} [{}/{} ({:.0f}%)]\tBoneAge Loss: {:.4f}\tGender Loss: {:.4f}\t Loss: {:.4f}\tCCC: {:.4f}'.format(
        epoch, batch_idx * len(data), num_samp,
        100. * batch_idx / num_samp, train_boneage_loss / num_samp,
        train_gender_loss / num_samp, train_loss / num_samp,
        train_ccc / num_samp))
    
#     print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.4f}\tCCC: {:.4f}'.format(
#         epoch, batch_idx * len(data), num_samp,
#         100. * batch_idx / num_samp, train_loss / num_samp, train_ccc / num_samp))

def test(test_loader):
    model.eval()
    test_avg_gender = 0
    test_boneage_loss = 0
    test_boneage_l1_loss = 0
    test_boneage_round_loss = 0
    test_boneage_round_loss_2 = 0
    test_boneage_round_loss_6 = 0
    test_gender_loss = 0
    test_loss = 0
    test_ccc = 0
    num_samp = len(test_loader.dataset)
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        print(sample['label'].int())

#         test_avg_gender += sample['label'][:,1].sum()
#         ba_output = model(data)
        
        # 2 heads MTL output
        ba_output, g_output = model(data)
#         print(ba_output)
        
#         print(output.size())
#         boneage_out = torch.round(output[:,0].div(6)).mul(6)
#         boneage_out = output[:,0]
#         print(boneage_out)
#         print(target[:,0])
#         boneage_loss = F.l1_loss(output.view(-1), target[:,0])

        boneage_loss = F.smooth_l1_loss(ba_output.view(-1), target[:,0])
        boneage_l1_loss = F.l1_loss(ba_output.view(-1), target[:,0])
# #         print(boneage_loss)
        boneage_round_loss = F.l1_loss(torch.round(ba_output.view(-1)), target[:,0])
# #         print(boneage_round_loss)
        boneage_round_loss_2 = F.l1_loss(torch.round(ba_output.view(-1).div(2)).mul(2), target[:,0])
# #         print(boneage_round_loss_2)
        boneage_round_loss_6 = F.l1_loss(torch.round(ba_output.view(-1).div(6)).mul(6), target[:,0])
# #         print(boneage_round_loss_6)
        
        test_boneage_loss += boneage_loss.data[0]
        test_boneage_l1_loss += boneage_l1_loss.data[0]
        test_boneage_round_loss += boneage_round_loss.data[0]
        test_boneage_round_loss_2 += boneage_round_loss_2.data[0]
        test_boneage_round_loss_6 += boneage_round_loss_6.data[0]
    
#         print(test_boneage_loss)
        gender_loss = F.binary_cross_entropy_with_logits(g_output.view(-1), target[:,1])
        test_gender_loss += gender_loss.data[0]

        loss = 0.75 * boneage_loss + 0.25 * gender_loss
#         loss = boneage_loss
#         loss = 0.8 * F.l1_loss(output.view(-1), target[:,0]) + 0.2 * F.binary_cross_entropy(F.sigmoid(output.view(-1)), target[:,1])
#         loss = F.l1_loss(output.view(-1), target[:,0])

        test_loss += loss.data[0]
#         c = ccc(boneage_out, target[:,0])
        c = ccc(ba_output, target[:,0])
        test_ccc += c.data.cpu().numpy()[0]

    print('\nTest set: BoneAge loss: {:.4f}, Gender loss: {:.4f}, loss: {:.4f}, CCC: {:.4f}'.format(
        test_boneage_loss / num_samp, test_gender_loss / num_samp,
        test_loss / num_samp, test_ccc / num_samp))
    print('\nTest set: L1 loss: {:.4f},Round loss: {:.4f}, Round2 loss: {:.4f}, Round6: {:.4f}, Gender: {:.4f}\n'.format(
        test_boneage_l1_loss / num_samp, test_boneage_round_loss / num_samp, test_boneage_round_loss_2 / num_samp,
        test_boneage_round_loss_6 / num_samp, test_avg_gender / num_samp))
    
#     print('\nTest set: Avg loss: {:.4f} CCC: {:.4f}\n'.format(
#         test_loss / num_samp, test_ccc / num_samp))

In [14]:
# torch.save(model.state_dict(), '../models/9-27-1540-resnet50-1fold-5.7889-2.56.pth')

In [ ]:
for epoch in range(1, 60 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])